In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

file_path = "./data/fraud_detection_data.feather"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data loading

In [7]:
df = pd.read_feather(file_path)
df.shape

(295758, 3)

In [8]:
df.head(5)

,custom_person_hash,link_value,link_type
0,00054865609df18bb5bfcf28f17538db,19830504,dt_birth
1,00054865609df18bb5bfcf28f17538db,jamielagaay@hotmail.com,email
2,00054865609df18bb5bfcf28f17538db,3071kr_160a,house_id
3,00054865609df18bb5bfcf28f17538db,NL54ABNA0434078468,iban
4,00054865609df18bb5bfcf28f17538db,lagaay,last_name


In [10]:
df["link_type"].value_counts(dropna=False)

link_type
email            42582
phone            40377
iban             39133
last_name        38557
dt_birth         37789
policy_driver    27476
policy_holder    27369
license_plate    26998
house_id         15477
Name: count, dtype: int64